<a href="https://colab.research.google.com/github/Miron-Hanukaiev/Introduction_to_Cloud_Computing/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

JSON_PATH = "/content/drive/MyDrive/students.json"
print("Using:", JSON_PATH)

Mounted at /content/drive
Using: /content/drive/MyDrive/students.json


In [5]:
# Load/Save helpers
import json
from typing import List, Dict

REQUIRED_FIELDS = ["name", "family_name", "email", "courses", "link"]
OPTIONAL_FIELD  = "favorite_show"

def load_students(path: str) -> List[Dict]:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for s in data:
        # keep required fields as-is
        s.setdefault("courses", [])
        # favorite_show is optional; default empty string for UI
        if OPTIONAL_FIELD not in s:
            s[OPTIONAL_FIELD] = ""
    return data

def save_students(students: List[Dict], path: str):
    # Persist only required fields + favorite_show if present
    cleaned = []
    for s in students:
        rec = {k: s[k] for k in REQUIRED_FIELDS if k in s}
        if s.get(OPTIONAL_FIELD, "") != "":
            rec[OPTIONAL_FIELD] = s[OPTIONAL_FIELD]
        else:
            # if empty, we keep it as empty to reflect user clearing the field
            rec[OPTIONAL_FIELD] = ""
        cleaned.append(rec)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(cleaned, f, ensure_ascii=False, indent=2)

students = load_students(JSON_PATH)
print(f"Loaded {len(students)} students.")


Loaded 2 students.


In [6]:
# Minimal UI with ipywidgets: select student, show read-only details, edit favorite_show, save back to JSON
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

def full_name(s):
    return f"{s['name']} {s['family_name']}"

# Widgets
student_dd   = widgets.Dropdown(
    options=[(full_name(s), i) for i, s in enumerate(students)],
    description="Student:",
    layout=widgets.Layout(width="320px")
)
name_tb      = widgets.Text(description="Name:", disabled=True, layout=widgets.Layout(width="320px"))
family_tb    = widgets.Text(description="Family:", disabled=True, layout=widgets.Layout(width="320px"))
email_tb     = widgets.Text(description="Email:", disabled=True, layout=widgets.Layout(width="520px"))
link_tb      = widgets.Text(description="Link:", disabled=True, layout=widgets.Layout(width="520px"))
courses_tb   = widgets.Textarea(description="Courses:", disabled=True, layout=widgets.Layout(width="650px", height="60px"))
fav_tb       = widgets.Text(description="Favorite show:", placeholder="Enter favorite show...", layout=widgets.Layout(width="400px"))
save_btn     = widgets.Button(description="Update", button_style="success")
msg_out      = widgets.Output()
card_out     = widgets.Output()

def fill_form(idx: int):
    s = students[idx]
    name_tb.value    = s.get("name","")
    family_tb.value  = s.get("family_name","")
    email_tb.value   = s.get("email","")
    link_tb.value    = s.get("link","")
    courses_tb.value = ", ".join(s.get("courses", []))
    fav_tb.value     = s.get("favorite_show","")
    render_card(s)

def render_card(s):
    with card_out:
        clear_output()
        html = f"""
        <div style="font-family:Arial;max-width:760px;border:1px solid #e5e7eb;border-radius:14px;padding:14px;
                    box-shadow:0 6px 16px rgba(0,0,0,.07); background: linear-gradient(135deg,#ffffff 0%,#f7fbff 100%);">
          <div style="font-weight:700;font-size:20px;margin-bottom:6px;">🎓 {s.get('name','')} {s.get('family_name','')}</div>
          <div style="color:#374151">{s.get('email','')}</div>
          <div><a href="{s.get('link','')}" target="_blank" style="color:#2563eb">{s.get('link','')}</a></div>
          <div style="margin-top:6px"><b>Courses:</b> {' · '.join(s.get('courses', []))}</div>
          <div style="margin-top:6px"><b>Favorite show:</b> {s.get('favorite_show','')}</div>
        </div>
        """
        display(HTML(html))

def on_student_change(change):
    if change["name"] == "value":
        fill_form(change["new"])

student_dd.observe(on_student_change)

def on_save_clicked(_):
    idx = student_dd.value
    students[idx]["favorite_show"] = fav_tb.value.strip()
    save_students(students, JSON_PATH)
    render_card(students[idx])
    with msg_out:
        clear_output()
        print("Updated favorite show and saved to students.json")

# Layout
save_btn.on_click(on_save_clicked)

ui = widgets.VBox([
    student_dd,
    widgets.HBox([name_tb, family_tb]),
    email_tb,
    link_tb,
    courses_tb,
    fav_tb,
    save_btn,
    msg_out,
    widgets.HTML("<hr>"),
    widgets.HTML("<b>Preview:</b>"),
    card_out
])

display(ui)
fill_form(0)
